# Joining University Data and GeoID + County IDs using Location Crosswalk

### Progress: 

## Pulling University Data and Patents Data From Box

In [1]:
from boxsdk import DevelopmentClient
import pandas as pd
import dotenv
import os
from io import BytesIO

In [2]:
env_location =  'C:\\Users\\trent\\Documents\\Capstone' ##change as necessaru
os.chdir(env_location)
dotenv.load_dotenv() # register the .env file where passwords are stored

True

In [3]:
CLIENT_ID = os.getenv("box_client_id")
CLIENT_SECRET = os.getenv("box_client_secret")
TOKEN = 'o6JpvozwmsE1vS7K7ukESbvw3qQXpY35' ##copy and paste from dev

In [4]:
from boxsdk import OAuth2, Client

oauth = OAuth2(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    access_token=TOKEN,
)

box = Client(oauth)

me = box.user().get()

print('logged into Box as', me.login)

logged into Box as txc5ve@virginia.edu


In [5]:
MY_FOLDER_ID = 0
my_folder = box.folder(MY_FOLDER_ID).get()
my_folder
items = my_folder.get_items()
items
for i in items:
    print(i)

<Box Folder - 166970315293 (Patents22)>


In [6]:
universities_box_id = '988549931799'
universities_file = box.file(universities_box_id).get()

In [7]:
universities_content = box.file(universities_box_id).content()

In [8]:
universities = pd.read_csv(BytesIO(universities_content))

In [9]:
## crosswalk not used for join
crosswalk_box_id = '990881584176'
crosswalk_file = box.file(crosswalk_box_id).get()
crosswalk_file
crosswalk_content = box.file(crosswalk_box_id).content()
crosswalk = pd.read_csv(BytesIO(crosswalk_content))

C:\Users\trent\AppData\Local\Temp\ipykernel_34284\1472077329.py:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  crosswalk = pd.read_csv(BytesIO(crosswalk_content))


In [ ]:
# patents_box_id = '987961636758'
# patents_file = box.file(patents_box_id).get()
# patents_file
# patents_content = box.file(patents_box_id).content()
# patents = pd.read_csv(BytesIO(patents_content))

In [10]:
universities

,Unnamed: 0,Institution,Control,City,State,level
0,0,Arizona State University Campus Immersion,Public,Tempe,AZ,r1
1,1,Auburn University,Public,Auburn,AL,r1
2,2,Baylor University,Private (non-profit),Waco,TX,r1
3,3,Binghamton University,Public,Vestal,NY,r1
4,4,Boston College,Private (non-profit),Chestnut Hill,MA,r1
...,...,...,...,...,...,...
297,18,University of Texas Health Science Center at H...,Public,Houston,TX,r3
298,19,University of Texas Health Science Center at S...,Public,San Antonio,TX,r3
299,20,University of Texas Medical Branch,Public,Galveston,TX,r3
300,21,University of Texas Southwestern Medical Center,Public,Dallas,TX,r3


In [11]:
crosswalk.head(5)

,id,GEOID,pv_city,gl_city,pv_state,gl_state,country,pv_lat,gl_lat,pv_long,gl_long,pv_county,gl_county,pv_state_fips,pv_county_fips,gl_county_fips
0,f9139cb2-cb8f-11eb-9615-121df0c29c1e,6075,San Francisco,NaN,CA,NaN,US,37.7292,NaN,-123.0470,NaN,San Francisco,NaN,6.0,6075.0,NaN
1,ffc9f8bd-cb8e-11eb-9615-121df0c29c1e,32031,Reno,NaN,NV,NaN,US,39.5504,NaN,-119.8030,NaN,Washoe,NaN,32.0,32031.0,NaN
2,9596257a-cb90-11eb-9615-121df0c29c1e,50021,Rutland,NaN,VT,NaN,US,43.6106,NaN,-72.9726,NaN,Rutland,NaN,50.0,50021.0,NaN
3,499f8deb-cb8e-11eb-9615-121df0c29c1e,34003,Parkridge,Park Ridge,NJ,NJ,US,NaN,-74.03939,NaN,41.03379,NaN,Bergen,34.0,NaN,34003.0
4,ea59a211-09be-11ec-893a-12de62d610b1,6085,Sunnyvale,NaN,CA,NaN,US,37.3688,NaN,-122.0360,NaN,Santa Clara,NaN,6.0,6085.0,NaN


## Cleaning Data for Join on City/State

In [12]:
universities.City = universities.City.str.lower()
universities.State = universities.State.str.lower()
universities.dtypes

Unnamed: 0      int64
Institution    object
Control        object
City           object
State          object
level          object
dtype: object

In [13]:
##crosswalk ultimately not used due to the typos in city names
crosswalk.pv_city = crosswalk.pv_city.str.lower()
crosswalk.pv_state = crosswalk.pv_state.str.lower()
crosswalk.dtypes
crosswalk.query("pv_city == 'chestnut hill'")

,id,GEOID,pv_city,gl_city,pv_state,gl_state,country,pv_lat,gl_lat,pv_long,gl_long,pv_county,gl_county,pv_state_fips,pv_county_fips,gl_county_fips
2777,e297d56d-cb8e-11eb-9615-121df0c29c1e,25017,chestnut hill,Newton,ma,MA,US,42.3268,-71.17208,-71.1646,42.33295,NaN,Middlesex,25.0,NaN,25017.0


In [14]:
# patents.pv_city = patents.pv_city.str.lower()
# patents

### Joining on City and State

#### Fix Missing Cities first

In [15]:
universities['City']= universities['City'].replace('saint louis', 'st. louis')
universities['City']= universities['City'].replace('mount pleasant', 'mt. pleasant')

## this one listed wrong in wikipedia table, rio grande valley is not a city
universities['City']= universities['City'].replace('rio grande valley', 'edinburg')

In [16]:
university_geo_join = pd.merge(universities, crosswalk, left_on=['City', 'State'], right_on=['pv_city','pv_state'], how='left')

In [17]:
university_geo = university_geo_join[['Institution', 'Control', 'City', 'State', 'level', 'GEOID', 'pv_city', 'pv_state', 'pv_county']]

In [18]:
university_geo

,Institution,Control,City,State,level,GEOID,pv_city,pv_state,pv_county
0,Arizona State University Campus Immersion,Public,tempe,az,r1,4013,tempe,az,Maricopa
1,Auburn University,Public,auburn,al,r1,1081,auburn,al,Lee
2,Baylor University,Private (non-profit),waco,tx,r1,48309,waco,tx,McLennan
3,Binghamton University,Public,vestal,ny,r1,36007,vestal,ny,NaN
4,Boston College,Private (non-profit),chestnut hill,ma,r1,25017,chestnut hill,ma,NaN
...,...,...,...,...,...,...,...,...,...
308,University of Texas Health Science Center at H...,Public,houston,tx,r3,48201,houston,tx,Harris
309,University of Texas Health Science Center at S...,Public,san antonio,tx,r3,48029,san antonio,tx,Bexar
310,University of Texas Medical Branch,Public,galveston,tx,r3,48167,galveston,tx,Galveston
311,University of Texas Southwestern Medical Center,Public,dallas,tx,r3,48113,dallas,tx,Dallas


In [19]:
university_geo[university_geo['GEOID'].isnull()]

,Institution,Control,City,State,level,GEOID,pv_city,pv_state,pv_county
222,Prairie View A&M University,Public,prairie view,tx,r2,NaN,NaN,NaN,NaN


In [20]:
###still an issiue with prairie view, not in cross walk but county;'s geoid is below

crosswalk.loc[crosswalk.pv_city.str.contains('prairie')].query("pv_county == 'tx'")

,id,GEOID,pv_city,gl_city,pv_state,gl_state,country,pv_lat,gl_lat,pv_long,gl_long,pv_county,gl_county,pv_state_fips,pv_county_fips,gl_county_fips


In [21]:
crosswalk.query("pv_county == 'Waller'")

,id,GEOID,pv_city,gl_city,pv_state,gl_state,country,pv_lat,gl_lat,pv_long,gl_long,pv_county,gl_county,pv_state_fips,pv_county_fips,gl_county_fips
1565,3d3f18f9-cb8e-11eb-9615-121df0c29c1e,48473,waller,NaN,tx,NaN,US,30.0602,NaN,-95.9256,NaN,Waller,NaN,48.0,48473.0,NaN
2502,b41951e7-09bc-11ec-893a-12de62d610b1,48473,hempstead,NaN,tx,NaN,US,30.1053,NaN,-96.0805,NaN,Waller,NaN,48.0,48473.0,NaN
10167,449aac0c-cb8e-11eb-9615-121df0c29c1e,48473,brookshire,NaN,tx,NaN,US,29.7793,NaN,-95.9540,NaN,Waller,NaN,48.0,48473.0,NaN
18058,83e12478-cb8e-11eb-9615-121df0c29c1e,48473,pattison,NaN,tx,NaN,US,29.8213,NaN,-95.9859,NaN,Waller,NaN,48.0,48473.0,NaN


In [22]:
# university_geo = university_geo[
    
university_geo.GEOID.fillna("48473", inplace = True)
university_geo.query("City == 'prairie view'")

C:\Users\trent\AppData\Local\Temp\ipykernel_34284\2182127261.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  university_geo.GEOID.fillna("48473", inplace = True)


,Institution,Control,City,State,level,GEOID,pv_city,pv_state,pv_county
222,Prairie View A&M University,Public,prairie view,tx,r2,48473,NaN,NaN,NaN


In [23]:
university_geo.query("City == 'prairie view'")

,Institution,Control,City,State,level,GEOID,pv_city,pv_state,pv_county
222,Prairie View A&M University,Public,prairie view,tx,r2,48473,NaN,NaN,NaN


In [24]:
university_geo[university_geo['GEOID'].isnull()]

,Institution,Control,City,State,level,GEOID,pv_city,pv_state,pv_county


In [26]:
# university_geo.loc[university_geo.GEOID.str.contains('_')]

### Group GEOIDS and get university count
#### First need to one hot encode to get a numerical count for each level

In [27]:
def one_hot_encode(df, col):
    one_hot = pd.get_dummies(df[col])
    one_hot.columns = [col + '_' + str(x) for x in one_hot.columns]
    
#     df = df.drop(col, axis=1)
    df = df.join(one_hot)
    
    return df


In [28]:
university_geo_levels = one_hot_encode(university_geo, 'level')

In [34]:
university_geo_levels

,Institution,Control,City,State,level,GEOID,pv_city,pv_state,pv_county,level_r1,level_r2,level_r3
0,Arizona State University Campus Immersion,Public,tempe,az,r1,4013,tempe,az,Maricopa,1,0,0
1,Auburn University,Public,auburn,al,r1,1081,auburn,al,Lee,1,0,0
2,Baylor University,Private (non-profit),waco,tx,r1,48309,waco,tx,McLennan,1,0,0
3,Binghamton University,Public,vestal,ny,r1,36007,vestal,ny,NaN,1,0,0
4,Boston College,Private (non-profit),chestnut hill,ma,r1,25017,chestnut hill,ma,NaN,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
308,University of Texas Health Science Center at H...,Public,houston,tx,r3,48201,houston,tx,Harris,0,0,1
309,University of Texas Health Science Center at S...,Public,san antonio,tx,r3,48029,san antonio,tx,Bexar,0,0,1
310,University of Texas Medical Branch,Public,galveston,tx,r3,48167,galveston,tx,Galveston,0,0,1
311,University of Texas Southwestern Medical Center,Public,dallas,tx,r3,48113,dallas,tx,Dallas,0,0,1


In [32]:
geoids_universities

,level_r1,level_r2,level_r3
GEOID,,,
1073,1,0,0
1081,1,0,0
1089,1,0,0
1097,0,1,0
1125,1,0,0
...,...,...,...
55079,1,1,1
56001,0,1,0
72127,0,1,0
